In [1]:
import sys
import os
import pandas as pd
import re
import warnings

## Data Processing

In [15]:
import pandas as pd
# df = pd.read_csv('/content/gdrive/MyDrive/telegram_data.csv')
df = pd.read_csv('../data/telegram_data.csv')
# Set display options to show full text
pd.set_option('display.max_colwidth')
pd.set_option('display.max_columns')

In [ ]:
def txtToConll(txt_file, conll_file):
    with open(txt_file, 'r', encoding='utf-8') as f:
        lines = f.readlines()

    with open(conll_file, 'w', encoding='utf-8') as f:
        for line in lines:
            # Assuming each line contains a sentence
            tokens = line.strip().split()  # Tokenize by whitespace
            
            # Example: Assign labels (customize this as necessary)
            # For demonstration, we're labeling all tokens as 'O'
            # Replace this logic with your actual label assignment
            for token in tokens:
                # Write each token with its label
                f.write(f"{token} O\n")  # Change 'O' to your actual label if needed
            
            # Add a blank line to separate sentences
            f.write("\n")

# Usage
txt_file = '../labeled_telegram_product_price_location.txt'  # Path to your input TXT file
conll_file = '../labeled_zemen_express.conll'  # Path to save your CoNLL file
txtToConll(txt_file, conll_file)

In [6]:
print("Checking for NaN values in the 'Message' column:")
nan_count = df['Message'].isnull().sum()
print(f"Number of NaN values in 'Message' column: {nan_count}")

Checking for NaN values in the 'Message' column:
Number of NaN values in 'Message' column: 956


In [4]:
df = df.dropna(subset=['Message'])

# Print the shape of the dataset after dropping NaN values in the "Message" column
print(f"Dataset shape after dropping NaN values in 'Message' column: {df.shape}")

Dataset shape after dropping NaN values in 'Message' column: (4044, 6)


In [ ]:
df = pd.read

- check the text

In [8]:
text1 = df['Message'].iloc[8]
# print(text1)

In [9]:
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [10]:
import re
def preprocess_text(text):

    text = re.sub(r'[^\w\s,]', '', text) # Remove emojis
    text = re.sub(r'\.{2,}', '', text) # Remove multiple dots
    text = re.sub(r'\s+', ' ', text).strip() # Remove extra spaces
    text = re.sub(r'\b[a-zA-Z]+\b', '', text) # Remove English words
    # text = re.sub(r'\b\d{10,}\b', '', text) # Remove phone numbers

    # Step 1: Identify and label prices
    def label_prices(text):
        # Find prices with the pattern like '200 ብር'
        prices = re.findall(r'\d+\s*ብር', text)
        # Replace identified prices with a label 'PRICE'
        for price in prices:
            text = text.replace(price, 'PRICE')
        return text,prices
    text_with_labeled_prices,prices = label_prices(text)

    # Step 2: Remove other numbers while keeping prices intact
    cleaned_text = re.sub(r'\b(?!PRICE\b)\d+\b', '', text_with_labeled_prices)
    # cleaned_text = re.sub(r'\b\S*\d+\S*\b', '', cleaned_text)  # Remove alphanumerics like 'S05/S06'

    # Step 3: Remove 'PRICE' label back to the original price format
    # cleaned_text = re.sub(r'\b\d+\b(?!\s*ብር)', '', text_with_labeled_prices)

    # Clean up extra spaces
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text).strip()

    # Replace the 'PRICE' label back to the original price format
    for price in prices:
        cleaned_text = cleaned_text.replace('PRICE', price)

    # Step 1: Text Cleaning
    cleaned_text = re.sub(r'[A-Za-z]', '', cleaned_text)  # Remove English letters
    cleaned_text = re.sub(r'[^\w\s]', '', cleaned_text)  # Remove punctuation
    cleaned_text = re.sub(r'\b(ቁ1|ቁ2)', '', cleaned_text) # Removing ቁ1 and ቁ2 from locations
    cleaned_text = cleaned_text.replace('_', ' ') # Replacing underscore with space
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text).strip()  # Remove extra spaces
    cleaned_text = cleaned_text.replace('0506', '') # Convert to lowercase text.replace('0506', '')
    # Step 2: Tokenization
    tokens = word_tokenize(cleaned_text)

    # Remove unwanted characters (keeping Amharic characters)
    # text = re.sub(r'[^አ-ፈ0-9\s]', '', text)  # Keep only Amharic characters and numbers
    return cleaned_text

In [11]:
preprocess_text(text1)
# cleaned_text

NameError: name 'word_tokenize' is not defined

In [42]:
len(tokens)

61

In [43]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform([' '.join(tokens)])
X.toarray()
X

<1x54 sparse matrix of type '<class 'numpy.float64'>'
	with 54 stored elements in Compressed Sparse Row format>

In [15]:
dft = df.copy()


In [16]:
dft['cleaned_message'] = dft['Message'].apply(preprocess_text)

- better so far

In [39]:
import re

def label_message_utf8_with_birr(message):
    # # Split the message at the first occurrence of '\n'
    # if '\n' in message:
    #     first_line, remaining_message = message.split('\n', 1)
    # else:
    #     first_line, remaining_message = message, ""

    labeled_tokens = []
    location_listB = [ 'ፒያሳ',  'መገናኛ']
    location_listI = ['መሰረት', 'ደፋር', 'ሞል', 'ጊዮርጊስ', 'አደባባይ', 'ራመት', 'ታቦር', 'ኦዳ', 'ህንፃ','ደራርቱ', 'ህንፃ']

    # # Tokenize the first line
    # first_line_tokens = re.findall(r'\S+', first_line)

    # # Label the first token as B-PRODUCT and the rest as I-PRODUCT
    # if first_line_tokens:
    #     labeled_tokens.append(f"{first_line_tokens[0]} B-PRODUCT")  # First token as B-PRODUCT
    #     for token in first_line_tokens[1:]:
    #         labeled_tokens.append(f"{token} I-PRODUCT")  # Remaining tokens as I-PRODUCT

    # Process the remaining message normally
    if message:
        lines = message.split('\n')
        for line in lines:
            tokens = re.findall(r'\S+', line)  # Tokenize each line while considering non-ASCII characters

            for token in tokens:
                # Check if token is a price (e.g., 500 ETB, $100, or ብር)
                if re.match(r'^\d{10,}$', token):
                    labeled_tokens.append(f"{token} O")  # Label as O for "other" or outside of any entity
                elif 'ዋጋ' in token:
                    labeled_tokens.append(f"{token} B-PRICE")
                # elif re.match(r'^\d+(\.\d{1,2})?$', token) and 'ብር' in token or 'ETB' in token or '$' in token or 'ብር' in token:
                #     labeled_tokens.append(f"{token} I-PRICE")
                # elif re.search(r'\b(ዋጋ|ብር)\b', token):
                #     # Extract the numeric part if it's a valid price
                #     price_match = re.search(r'(?<!\d)(0|[1-9]\d*)(\.\d{1,2})?', token)
                #     if price_match:
                #         labeled_tokens.append(f"{price_match.group()} I-PRICE")  # Label the price
                #         # numbers_only.append(price_match.group())  # Store the number only
                # Check if token could be a location (e.g., cities or general location names)
                elif re.match(r'^\d+(\.\d{1,2})?$', token) or 'ብር' in token:
                    labeled_tokens.append(f"{token} I-PRICE")
                elif re.match(r'^\d+(\.\d{1,2})?\s*ብር$', token):
                    labeled_tokens.append(f"{token} I-PRICE")
                elif 'ብር' in token:
                    labeled_tokens.append(f"{token} I-PRICE")
                elif any(loc in token for loc in location_listB):
                    labeled_tokens.append(f"{token} B-LOC")
                elif any(loc in token for loc in location_listI):
                    labeled_tokens.append(f"{token} I-LOC")
                # Assume other tokens are part of a product name or general text
                else:
                    labeled_tokens.append(f"{token} O")

    return "\n".join(labeled_tokens)






In [40]:
# Apply the updated function to the non-null messages
dft['Labeled_Message'] = dft['cleaned_message'].apply(label_message_utf8_with_birr)

In [41]:
# Display the updated DataFrame
dft[['cleaned_message', 'Labeled_Message']].head()

,cleaned_message,Labeled_Message
0,የሞተ ቆዳን እንዲሁም ቆሻሻን ለማፅዳት ተመራጭ ዋጋ 200 ብር ውስን ፍሬ ነው ያለው አድራሻ መገናኛ መሰረት ደፋር ሞል ሁለተኛ ፎቅ ቢሮ ቁ ፒያሳ ጊዮርጊስ አደባባይ ራመት ታቦር ኦዳ ህንፃ 1ኛ ፎቅ ሱቅ ቁ 1 ፒያሳ ቅርንጫፍ በ ለማዘዝ ይጠቀሙ ለተጨማሪ ማብራሪያ የቴሌግራም ገፃችን,የሞተ O\nቆዳን O\nእንዲሁም O\nቆሻሻን O\nለማፅዳት O\nተመራጭ O\nዋጋ B-PRICE\n200 I-PRICE\nብር I-PRICE\nውስን O\nፍሬ O\nነው O\nያለው O\nአድራሻ O\nመገናኛ B-LOC\nመሰረት I-LOC\nደፋር I-LOC\nሞል I-LOC\nሁለተኛ O\nፎቅ O\nቢሮ O\nቁ O\nፒያሳ B-LOC\nጊዮርጊስ I-LOC\nአደባባይ I-LOC\nራመት I-LOC\nታቦር I-LOC\nኦዳ I-LOC\nህንፃ I-LOC\n1ኛ O\nፎቅ O\nሱቅ O\nቁ O\n1 I-PRICE\nፒያሳ B-LOC\nቅርንጫፍ O\nበ O\nለማዘዝ O\nይጠቀሙ O\nለተጨማሪ O\nማብራሪያ O\nየቴሌግራም O\nገፃችን O
1,የሞተ ቆዳን እንዲሁም ቆሻሻን ለማፅዳት ተመራጭ ዋጋ 200 ብር ውስን ፍሬ ነው ያለው አድራሻ መገናኛ መሰረት ደፋር ሞል ሁለተኛ ፎቅ ቢሮ ቁ ፒያሳ ጊዮርጊስ አደባባይ ራመት ታቦር ኦዳ ህንፃ 1ኛ ፎቅ ሱቅ ቁ 1 ፒያሳ ቅርንጫፍ በ ለማዘዝ ይጠቀሙ ለተጨማሪ ማብራሪያ የቴሌግራም ገፃችን,የሞተ O\nቆዳን O\nእንዲሁም O\nቆሻሻን O\nለማፅዳት O\nተመራጭ O\nዋጋ B-PRICE\n200 I-PRICE\nብር I-PRICE\nውስን O\nፍሬ O\nነው O\nያለው O\nአድራሻ O\nመገናኛ B-LOC\nመሰረት I-LOC\nደፋር I-LOC\nሞል I-LOC\nሁለተኛ O\nፎቅ O\nቢሮ O\nቁ O\nፒያሳ B-LOC\nጊዮርጊስ I-LOC\nአደባባይ I-LOC\nራመት I-LOC\nታቦር I-LOC\nኦዳ I-LOC\nህንፃ I-LOC\n1ኛ O\nፎቅ O\nሱቅ O\nቁ O\n1 I-PRICE\nፒያሳ B-LOC\nቅርንጫፍ O\nበ O\nለማዘዝ O\nይጠቀሙ O\nለተጨማሪ O\nማብራሪያ O\nየቴሌግራም O\nገፃችን O
2,ከብረት የተሰራ ጫማ እና የተለያዩ ዕቃወች ማስቀመጫ ለመገጣጠም ቀላል ባለ ስድስት መደርደሪያ ያለው ዋጋ 2250 ብር አድራሻ መገናኛ መሰረት ደፋር ሞል ሁለተኛ ፎቅ ቢሮ ቁ ፒያሳ ጊዮርጊስ አደባባይ ራመት ታቦር ኦዳ ህንፃ 1ኛ ፎቅ ሱቅ ቁ 1 ፒያሳ ቅርንጫፍ በ ለማዘዝ ይጠቀሙ ለተጨማሪ ማብራሪያ የቴሌግራም ገፃችን,ከብረት O\nየተሰራ O\nጫማ O\nእና O\nየተለያዩ O\nዕቃወች O\nማስቀመጫ O\nለመገጣጠም O\nቀላል O\nባለ O\nስድስት O\nመደርደሪያ O\nያለው O\nዋጋ B-PRICE\n2250 I-PRICE\nብር I-PRICE\nአድራሻ O\nመገናኛ B-LOC\nመሰረት I-LOC\nደፋር I-LOC\nሞል I-LOC\nሁለተኛ O\nፎቅ O\nቢሮ O\nቁ O\nፒያሳ B-LOC\nጊዮርጊስ I-LOC\nአደባባይ I-LOC\nራመት I-LOC\nታቦር I-LOC\nኦዳ I-LOC\nህንፃ I-LOC\n1ኛ O\nፎቅ O\nሱቅ O\nቁ O\n1 I-PRICE\nፒያሳ B-LOC\nቅርንጫፍ O\nበ O\nለማዘዝ O\nይጠቀሙ O\nለተጨማሪ O\nማብራሪያ O\nየቴሌግራም O\nገፃችን O
3,ዘመናዊ የልብስ ማስቀመጫ ቁምሳጥን በቀላሉ የሚገጣጠም የሚነቃቀል ከቦታ ወደ ቦታ ለማንቀሳቀስ የሚመች ቀላል ክብደት ያለው ባለ ዚፕ የሸራ ልብስ ያለው ፍሬሞቹ የብረት ቱቦዎች የሆኑ መገጣጠሚያዎቹ ጠንካራ ካርቦኔትድ ፕላስቲኮች የሆኑ ለቤትዎ ውበት የሚሠጥ ቦታ የማይዝ ሲሆን ቁመት ሳሜ የጎን ስፋት ሳሜ ወርድ ሳሜ ዋጋ 3500 ብር ውስን ፍሬ ነው ያለው አድራሻ መገናኛ መሰረት ደፋር ሞል ሁለተኛ ፎቅ ቢሮ ቁ ፒያሳ ጊዮርጊስ አደባባይ ራመት ታቦር ኦዳ ህንፃ 1ኛ ፎቅ ሱቅ ቁ 1 ፒያሳ ቅርንጫፍ በ ለማዘዝ ይጠቀሙ ለተጨማሪ ማብራሪያ የቴሌግራም ገፃችን,ዘመናዊ O\nየልብስ O\nማስቀመጫ O\nቁምሳጥን O\nበቀላሉ O\nየሚገጣጠም O\nየሚነቃቀል O\nከቦታ O\nወደ O\nቦታ O\nለማንቀሳቀስ O\nየሚመች O\nቀላል O\nክብደት O\nያለው O\nባለ O\nዚፕ O\nየሸራ O\nልብስ O\nያለው O\nፍሬሞቹ O\nየብረት O\nቱቦዎች O\nየሆኑ O\nመገጣጠሚያዎቹ O\nጠንካራ O\nካርቦኔትድ O\nፕላስቲኮች O\nየሆኑ O\nለቤትዎ O\nውበት O\nየሚሠጥ O\nቦታ O\nየማይዝ O\nሲሆን O\nቁመት O\nሳሜ O\nየጎን O\nስፋት O\nሳሜ O\nወርድ O\nሳሜ O\nዋጋ B-PRICE\n3500 I-PRICE\nብር I-PRICE\nውስን O\nፍሬ O\nነው O\nያለው O\nአድራሻ O\nመገናኛ B-LOC\nመሰረት I-LOC\nደፋር I-LOC\nሞል I-LOC\nሁለተኛ O\nፎቅ O\nቢሮ O\nቁ O\nፒያሳ B-LOC\nጊዮርጊስ I-LOC\nአደባባይ I-LOC\nራመት I-LOC\nታቦር I-LOC\nኦዳ I-LOC\nህንፃ I-LOC\n1ኛ O\nፎቅ O\nሱቅ O\nቁ O\n1 I-PRICE\nፒያሳ B-LOC\nቅርንጫፍ O\nበ O\nለማዘዝ O\nይጠቀሙ O\nለተጨማሪ O\nማብራሪያ O\nየቴሌግራም O\nገፃችን O
8,ዋጋ 150 ብር ውስን ፍሬ ነው ያለው አድራሻ መገናኛ መሰረት ደፋር ሞል ሁለተኛ ፎቅ ቢሮ ቁ ፒያሳ ጊዮርጊስ አደባባይ ራመት ታቦር ኦዳ ህንፃ 1ኛ ፎቅ ሱቅ ቁ 1 ፒያሳ ቅርንጫፍ በ ለማዘዝ ይጠቀሙ ለተጨማሪ ማብራሪያ የቴሌግራም ገፃችን,ዋጋ B-PRICE\n150 I-PRICE\nብር I-PRICE\nውስን O\nፍሬ O\nነው O\nያለው O\nአድራሻ O\nመገናኛ B-LOC\nመሰረት I-LOC\nደፋር I-LOC\nሞል I-LOC\nሁለተኛ O\nፎቅ O\nቢሮ O\nቁ O\nፒያሳ B-LOC\nጊዮርጊስ I-LOC\nአደባባይ I-LOC\nራመት I-LOC\nታቦር I-LOC\nኦዳ I-LOC\nህንፃ I-LOC\n1ኛ O\nፎቅ O\nሱቅ O\nቁ O\n1 I-PRICE\nፒያሳ B-LOC\nቅርንጫፍ O\nበ O\nለማዘዝ O\nይጠቀሙ O\nለተጨማሪ O\nማብራሪያ O\nየቴሌግራም O\nገፃችን O


In [129]:
# Check for entries that contain any numbers
has_numbers = dft['cleaned_message'].str.contains(r'\d', na=False)

# Count unique entries that contain numbers
unique_number_count = has_numbers.sum()
print("Count of unique entries with numbers in cleaned_message:", unique_number_count)

Count of unique entries with numbers in cleaned_message: 3785


In [27]:
# Select the specific row and column
row_index = 681  # For example, the second row
selected_column = 'cleaned_message'

# Get the text from the selected cell
text = dft.loc[row_index, selected_column]

# Count numbers using regex
numbers = re.findall(r'\d+', text)  # Find all occurrences of numbers
number_count = len(numbers)  # Count how many numbers are found

print(f"Text in selected row and column: {text}")
print(f"Number of numbers present: {number_count}")

Text in selected row and column: ዋጋ ርዝመት 160 500 ብር ውስን ፍሬ ነው ያለው አድራሻ መገናኛ መሰረት ደፋር ሞል ሁለተኛ ፎቅ ቢሮ ቁ  ፒያሳ ጊዮርጊስ አደባባይ ራመት ታቦር ኦዳ ህንፃ 1ኛ ፎቅ ሱቅ ቁ 1 ፒያሳ ቅርንጫፍ በ ለማዘዝ ይጠቀሙ ለተጨማሪ ማብራሪያ የቴሌግራም ገፃችን
Number of numbers present: 4


In [45]:
# Save the updated labeled dataset to a file in CoNLL format
labeled_data_birr_path = 'labeled_telegram_product_price_location.conll'
with open(labeled_data_birr_path, 'w', encoding='utf-8') as f:
    for index, row in dft.iterrows():
        f.write(f"{row['Labeled_Message']}\n\n")

### Tokenizer

In [ ]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline
tokenizer = AutoTokenizer.from_pretrained("masakhane/afroxlmr-large-ner-masakhaner-1.0_2.0")
model = AutoModelForTokenClassification.from_pretrained("masakhane/afroxlmr-large-ner-masakhaner-1.0_2.0")
nlp = pipeline("ner", model=model, tokenizer=tokenizer)
example = df['Message'][10]
ner_results = nlp(example)
print(ner_results)

[{'entity': 'B-LOC', 'score': 0.77822363, 'index': 43, 'word': '▁ስ', 'start': 117, 'end': 118}, {'entity': 'I-LOC', 'score': 0.58282, 'index': 44, 'word': 'ሪ', 'start': 118, 'end': 119}, {'entity': 'I-LOC', 'score': 0.8872894, 'index': 45, 'word': '▁', 'start': 120, 'end': 121}, {'entity': 'I-LOC', 'score': 0.7864612, 'index': 46, 'word': 'ኤም', 'start': 120, 'end': 122}, {'entity': 'I-LOC', 'score': 0.8502413, 'index': 47, 'word': '▁ሲ', 'start': 123, 'end': 124}, {'entity': 'I-LOC', 'score': 0.5737138, 'index': 48, 'word': 'ቲ', 'start': 124, 'end': 125}, {'entity': 'I-LOC', 'score': 0.5543835, 'index': 49, 'word': '▁ሞ', 'start': 126, 'end': 127}, {'entity': 'I-LOC', 'score': 0.8828338, 'index': 50, 'word': 'ል', 'start': 127, 'end': 128}, {'entity': 'I-LOC', 'score': 0.76899004, 'index': 51, 'word': '▁', 'start': 130, 'end': 131}, {'entity': 'I-LOC', 'score': 0.5774763, 'index': 52, 'word': 'ሁለተኛ', 'start': 130, 'end': 134}, {'entity': 'B-LOC', 'score': 0.9574409, 'index': 64, 'word': '

In [9]:
df.to_csv('telegram_data.csv', index=False)

In [ ]:
df.to_csv('labeled_data.csv')